# MIPath Vignette

### Package installation

Install MIPath with

`pip install mipathway`

# Example use case

Here, we will give an example of how to run a simple pathway analysis using **MIPath**.

## Acquiring and preprocessing the data

### Gene expression data

For this example, we will use a single cell dataset available at [EBI Single Cell Expression Atlas](https://www.ebi.ac.uk/gxa/sc/home), more particularly a dataset on **haematopoiesis**, taken from the following publication:

Ranzoni AM, Tangherloni A, Berest I, Riva SG, Myers B et al. (2021) Integrative Single-Cell RNA-Seq and ATAC-Seq Analysis of Human Developmental Hematopoiesis

The package **Scanpy** is very useful for this purpose, and can be installed using:

`pip install scanpy`

We first import both **MIPath** and **Scanpy**


In [1]:
import mipath as mp
import scanpy as sc

We then use Scanpy to download the dataset directly from the Single Cell Expression Atlas, by using its accession number *E-MTAB-9067*

In [2]:
dataset_id = 'E-MTAB-9067'

anndata = sc.datasets.ebi_expression_atlas(dataset_id)

/home/gus/envs/mitest2/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


We will now preprocess the data, by filtering out cells with less than 300 expressed genes, and genes that are expressed in less than 3 cells.

We then normalize the library sizes of each cell.

In [3]:
# Filtering
sc.pp.filter_cells(anndata, min_genes=200)
sc.pp.filter_genes(anndata, min_cells=3)

# Library size correction
sc.pp.normalize_total(anndata, target_sum=1e4)

Scanpy uses [anndata](https://anndata.readthedocs.io/en/latest/) data structures. We want to extract pandas DataFrames from it, which can be used as inputs to **MIPath**. 
More specifically, we need one DataFrame containing the gene expression information, and one with the metadata that we want to compare against.

In [4]:
metadata = anndata.obs

exp_data = anndata.to_df()

### Pathway annotation data

We also need a source of pathway annotation. This is most commonly done by using a **gmt** annotation file, such as the ones available at [MSigDB](https://www.gsea-msigdb.org/gsea/msigdb). 

One such file is included in this repo. This file however uses **HUGO gene names**, and we would like to use **Ensembl** annotation, as it matches our data (more on that later), so we provide the function with at dictionary that translates gene IDs for HUGO to Ensembl, created in [BioMart](https://www.ensembl.org/info/data/biomart/index.html).

In [5]:
kegg_genesets = mp.parse_gmt(gmt_path = './data/c2.cp.kegg.v7.5.1.symbols.gmt', gene_id_dict_path = './data/name2ensembl_human.txt')

Conveniently, **MIPath** also includes a function to download pathway annotation directly from [Reactome](https://www.reactome.org), simply by specifying the organism, and gene annotation scheme.

In [6]:
reactome_genesets = mp.get_reactome(organism = 'HSA', gene_anot = 'Ensembl')

### Structure of the data

Now that we have all the data we need, let's have a look at its structure, starting with the *expression data*:

In [7]:
exp_data.head(5)

,ENSG00000000003,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,...,ENSG00000289685,ENSG00000289690,ENSG00000289692,ENSG00000289694,ENSG00000289695,ENSG00000289697,ENSG00000289700,ENSG00000289701,ENSG00000289716,ENSG00000289718
ERR4147602,0.0,0.131484,0.00000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.131484,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
ERR4147603,0.0,0.000000,0.00000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.226321,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
ERR4147604,0.0,1.932760,0.00000,0.00000,0.0,0.000000,0.000000,0.101724,0.0,0.203448,...,4.830823,0.0,0.0,0.855688,0.0,0.0,0.0,0.0,0.0,0.0
ERR4147605,0.0,0.546372,0.09934,0.04967,0.0,0.149011,0.397362,0.049670,0.0,0.198681,...,0.320914,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
ERR4147606,0.0,1.001502,0.00000,0.00000,0.0,6.609936,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


We see that we have as row names individual IDs for each sample, and as column names gene IDs.

Let's now see the *metadata*:

In [8]:
metadata.head(5)

,Sample Characteristic[organism],Sample Characteristic Ontology Term[organism],Sample Characteristic[individual],Sample Characteristic Ontology Term[individual],Sample Characteristic[developmental stage],Sample Characteristic Ontology Term[developmental stage],Sample Characteristic[age],Sample Characteristic Ontology Term[age],Sample Characteristic[sex],Sample Characteristic Ontology Term[sex],...,Factor Value Ontology Term[single cell identifier],Factor Value[organism part],Factor Value Ontology Term[organism part],Factor Value[sampling site],Factor Value Ontology Term[sampling site],Factor Value[inferred cell type - authors labels],Factor Value Ontology Term[inferred cell type - authors labels],Factor Value[inferred cell type - ontology labels],Factor Value Ontology Term[inferred cell type - ontology labels],n_genes
ERR4147602,Homo sapiens,http://purl.obolibrary.org/obo/NCBITaxon_9606,Sample2,NaN,late embryonic stage,http://purl.obolibrary.org/obo/UBERON_0007220,17 week,NaN,female,http://purl.obolibrary.org/obo/PATO_0000383,...,NaN,bone marrow,http://purl.obolibrary.org/obo/UBERON_0002371,hip,http://purl.obolibrary.org/obo/UBERON_0001464,NaN,NaN,NaN,NaN,2191
ERR4147603,Homo sapiens,http://purl.obolibrary.org/obo/NCBITaxon_9606,Sample2,NaN,late embryonic stage,http://purl.obolibrary.org/obo/UBERON_0007220,17 week,NaN,female,http://purl.obolibrary.org/obo/PATO_0000383,...,NaN,bone marrow,http://purl.obolibrary.org/obo/UBERON_0002371,hip,http://purl.obolibrary.org/obo/UBERON_0001464,NaN,NaN,NaN,NaN,1810
ERR4147604,Homo sapiens,http://purl.obolibrary.org/obo/NCBITaxon_9606,Sample2,NaN,late embryonic stage,http://purl.obolibrary.org/obo/UBERON_0007220,17 week,NaN,female,http://purl.obolibrary.org/obo/PATO_0000383,...,NaN,bone marrow,http://purl.obolibrary.org/obo/UBERON_0002371,hip,http://purl.obolibrary.org/obo/UBERON_0001464,erythroid progenitor cell,http://purl.obolibrary.org/obo/CL_0000038,erythroid progenitor cell,http://purl.obolibrary.org/obo/CL_0000038,3929
ERR4147605,Homo sapiens,http://purl.obolibrary.org/obo/NCBITaxon_9606,Sample2,NaN,late embryonic stage,http://purl.obolibrary.org/obo/UBERON_0007220,17 week,NaN,female,http://purl.obolibrary.org/obo/PATO_0000383,...,NaN,bone marrow,http://purl.obolibrary.org/obo/UBERON_0002371,hip,http://purl.obolibrary.org/obo/UBERON_0001464,NaN,NaN,NaN,NaN,5115
ERR4147606,Homo sapiens,http://purl.obolibrary.org/obo/NCBITaxon_9606,Sample2,NaN,late embryonic stage,http://purl.obolibrary.org/obo/UBERON_0007220,17 week,NaN,female,http://purl.obolibrary.org/obo/PATO_0000383,...,NaN,bone marrow,http://purl.obolibrary.org/obo/UBERON_0002371,hip,http://purl.obolibrary.org/obo/UBERON_0001464,NaN,NaN,NaN,NaN,2137


We see that we have a lot of information on our samples. Most importantly, **the row names match the ones in the expression data**.

We also have many columns, and we do not want to use all of them. This is OK, we will specify the which columns to use later. Importantly, the columns we want to use contain *factors* that separate the samples, and not continuous data. If we want to analyse continuous metadata, it has to be discretized first in some way.

We can also look at how the geneset DataFrame looks like:

In [9]:
kegg_genesets.head(5)

,genes,url
KEGG_N_GLYCAN_BIOSYNTHESIS,"[ENSG00000284501, ENSG00000168282, ENSG0000003...",http://www.gsea-msigdb.org/gsea/msigdb/cards/K...
KEGG_OTHER_GLYCAN_DEGRADATION,"[ENSG00000262446, ENSG00000277926, ENSG0000022...",http://www.gsea-msigdb.org/gsea/msigdb/cards/K...
KEGG_O_GLYCAN_BIOSYNTHESIS,"[ENSG00000288490, ENSG00000158470, ENSG0000011...",http://www.gsea-msigdb.org/gsea/msigdb/cards/K...
KEGG_GLYCOSAMINOGLYCAN_DEGRADATION,"[ENSG00000127415, ENSG00000010404, ENSG0000016...",http://www.gsea-msigdb.org/gsea/msigdb/cards/K...
KEGG_GLYCOSAMINOGLYCAN_BIOSYNTHESIS_KERATAN_SULFATE,"[ENSG00000033170, ENSG00000175264, ENSG0000008...",http://www.gsea-msigdb.org/gsea/msigdb/cards/K...


It has pathway IDs as row names, and a column named *genes* that contains a list of genes in that pathway. This is all done by the parsing function.

With all the data loaded, we can proceed with the MIPath analysis.

## MIPath analysis

The MIPath analysis is done in two separate steps:
* **Decomposition**: decomposing the activity of a pathway in each cell by grouping cells with similar activities, and
* **Scoring**: comparing the decomposed pathway activities with factors present in the metadata.

The decomposition step is by far the most computationally intensive, and so it was kept separate, as its results can be saved and reused for different factors.

### Decomposition step

We use the `decompose_pathways` function, providing the expression data and pathway annotation:

In [10]:
decomposed_df = mp.decompose_pathways(data = exp_data, gene_sets_df = kegg_genesets)

Performing pathway decomposition
(1/186) --- KEGG_N_GLYCAN_BIOSYNTHESIS --- 46 genes
Computed 25 NN in 8.7 s
SNN scores in 0.1 s
Found 27 partitions in 1.0 s

(2/186) --- KEGG_OTHER_GLYCAN_DEGRADATION --- 24 genes
Computed 25 NN in 0.3 s
SNN scores in 0.1 s
Found 26 partitions in 0.5 s

(3/186) --- KEGG_O_GLYCAN_BIOSYNTHESIS --- 30 genes
Computed 25 NN in 0.3 s
SNN scores in 0.1 s
Found 30 partitions in 0.7 s

(4/186) --- KEGG_GLYCOSAMINOGLYCAN_DEGRADATION --- 20 genes
Computed 25 NN in 0.3 s
SNN scores in 0.1 s
Found 30 partitions in 0.7 s

(5/186) --- KEGG_GLYCOSAMINOGLYCAN_BIOSYNTHESIS_KERATAN_SULFATE --- 14 genes
Computed 25 NN in 0.3 s
SNN scores in 0.1 s
Found 34 partitions in 1.0 s

(6/186) --- KEGG_GLYCEROLIPID_METABOLISM --- 57 genes
Computed 25 NN in 0.3 s
SNN scores in 0.1 s
Found 30 partitions in 1.0 s

(7/186) --- KEGG_GLYCOSYLPHOSPHATIDYLINOSITOL_GPI_ANCHOR_BIOSYNTHESIS --- 25 genes
Computed 25 NN in 0.3 s
SNN scores in 0.1 s
Found 26 partitions in 1.1 s

(8/186) --- KEGG

It gives us a new DataFrame, with the cells grouped by the activity of each pathway.

### Scoring step

We can now finally score the pathways. In this example, we want to analyse which pathways are most associated with different **cell types**, so we will use the `Factor Value[inferred cell type - authors labels]` column in our metadata. We are also curious to see if there is any pathway in this case that is associated with the different **sampling sites**, which can be found in the `Factor Value[sampling site]` column.

In [11]:
factors_to_score = ['Factor Value[inferred cell type - authors labels]', 'Factor Value[sampling site]']

result = mp.score_factors(decomposed_df = decomposed_df, metadata = metadata, factors = factors_to_score)

Scoring Factor Value[inferred cell type - authors labels]
Scoring Factor Value[sampling site]


### Results

We can now display the results, sorting it by which pathways recieved the highest score for the **cell type**:

In [12]:
result.sort_values('Factor Value[inferred cell type - authors labels]', ascending=False).head(10)

,Factor Value[inferred cell type - authors labels],Factor Value[sampling site]
KEGG_HEMATOPOIETIC_CELL_LINEAGE,0.326898,0.024368
KEGG_ANTIGEN_PROCESSING_AND_PRESENTATION,0.282982,0.042522
KEGG_LYSOSOME,0.262620,0.014868
KEGG_CYTOKINE_CYTOKINE_RECEPTOR_INTERACTION,0.256345,0.010223
KEGG_CELL_ADHESION_MOLECULES_CAMS,0.231899,0.048265
KEGG_JAK_STAT_SIGNALING_PATHWAY,0.213601,0.010626
KEGG_PRIMARY_IMMUNODEFICIENCY,0.205740,0.008597
KEGG_SYSTEMIC_LUPUS_ERYTHEMATOSUS,0.203573,0.031544
KEGG_B_CELL_RECEPTOR_SIGNALING_PATHWAY,0.197873,0.015963
KEGG_TYPE_I_DIABETES_MELLITUS,0.193569,0.041785


We see that the pathway **KEGG_HEMATOPOIETIC_CELL_LINEAGE** has the highest score, which makes much sense.